# 02-01 - Calculating shortest paths and diameter

*April 24 2024*

Paths are a key concept in network analysis and graph learning. Paths tell us how nodes can indirectly influence each other via sequences of links, influence the detection of community structures and are the basis for message passing algorithms used in graph neural networks. In this notebook, we explore three basic algorithms to calculate shortest paths in networks. We further explore the interpretation of adjacency matrix powers and show how they can be used to calculate the diameter of a graph in a purely "algebraic" fashion.

In [6]:
from collections import defaultdict

import numpy as np
from scipy import linalg

import pathpyG as pp

# Dijkstra's algorithm for single-source shortest paths

In the lecture we mentioned Dikstra's algorithm to calculate the shortest from a given (single) source node to all other nodes. Here we provide a simple (and inefficient) implementation, which should only serve to refresh your memory of the algorithm.

The algorithm works as follows: Starting from the source node, we first set the distance to all other nodes to infinity. The distance to the source node itself is set to zero. We then add all nodes along with their distances to a set `Q` (which - for reasons of efficiency - is usually implemented as a priority queue). In each iteration of the algorithm, a node $u$ with minimal distance from the given source node is removed from `Q`. In the beginning, this is necessarily the source node, which is the only node with distance zero. In each iteration, the distance to all successors $v$ of node $u$ (i.e. nodes for which a link $(u,v)$ exists) is updated. If the old distance from the source node was larger than the distance from the source to $u$ plus one, we found a shorter path to $v$ that passes through node $u$. In the first step of the algorithm, this will update the distance of all neighbours of the source node to one. Repeating this loop will incrementally set the correct shortest path lengths along all paths starting from source. The algorithm is repeated until the set `Q` is empty. 

Whenever we update the distance of a node $v$, we can also store a pointer that points in the (opposite) direction of the shortest path from the source to node $v$. This will allow us to reconstruct all shortest paths and to visualise the shortest path structure of the network (for a single source) as a directed network.

In terms of computational complexity, we highlight that the implementation of the set `Q` is crucial. In the implementation below we simply use a dictionary to store the current distances of all nodes in `Q`. In each iteration we have to determine the key with minimal distance, which requires a linear search. This leads to a time complexity of $\mathcal{O}(n^2)$, where $n$ is the number of nodes in the network. An optimal implementation using a priority queue implemented as a Fibonacci heap has a worst-time complexity of $\mathcal{O}(n\log n+m)$, where $n$ is the number of nodes and $m$ is the number of edges.

In [13]:
def dijkstra(network: pp.Graph, source: str):
    """Naive (and inefficient) implementation of Dijkstra's algorithm 
    for unweighted networks"""
    
    Q: dict = dict()
    dist: dict = dict()
    prev: dict = dict()

    # Initialise distances and shortest path pointer for all other nodes
    for v in network.nodes:
        if v != source:
            dist[v] = np.inf
            prev[v] = None
        else:
            dist[source] = 0
        # add all nodes to Q
        Q[v] = dist[v]

    while Q: # Reeat until Q is empty

        # Find entry with minimal distance and remove from Q
        u = min(Q.keys(), key=(lambda k: Q[k])) # To do this more efficiently, we should actually use a priority queue
        del Q[u]

        # Calculate new distance values of all successors of u
        for v in network.successors(u):            
            
            # Here we could also use edge weights to calculate least cost paths
            new_dist = dist[u] + 1           
            
            # Update distance values and pointers if distance is smaller than previously known
            if new_dist < dist[v]:
                dist[v] = new_dist
                prev[v] = u
                if v in Q:
                    Q[v] = new_dist

    return dist, prev

We use the simple example undirected network, which we already know from the lecture.

In [16]:
n_undirected = pp.Graph.from_edge_list([('a', 'b'), ('b', 'c'), ('c', 'a'), ('d', 'e'), ('e', 'f'), ('f', 'g'), ('g', 'd'), ('d', 'f'), ('b', 'd')]).to_undirected()
pp.plot(n_undirected, edge_color='white', node_label = [x for x in n_undirected.mapping.node_ids]);

Let us try this for our undirected example network, using node `a` as the source node.

In [14]:
dist, prev = dijkstra(n_undirected, 'a')
print(dist)

{'a': 0, 'b': 1, 'c': 1, 'd': 2, 'e': 3, 'f': 3, 'g': 3}


We can use the pointer dictionary `prev` to generate a shortest path tree, i.e. an acyclic directed network that can be used to construct shortest paths from the given source node to any other node.

In [15]:
edges = []

for k, v in prev.items():
    if v != None:
        edges.append((v, k))
n_tree = pp.Graph.from_edge_list(edges)
pp.plot(n_tree, edge_color='white', node_label = [x for x in n_tree.mapping.node_ids])

Naturally, Dijkstra's algorithm works for undirected and directed networks:

In [17]:
n_directed = pp.Graph.from_edge_list([('a', 'b'), ('b', 'c'), ('c', 'a'), ('d', 'e'), ('e', 'f'), ('f', 'g'), ('g', 'd'), ('d', 'f'), ('b', 'd')])
pp.plot(n_directed, edge_color='white', node_label = [x for x in n_undirected.mapping.node_ids]);

In [18]:
dist, prev = dijkstra(n_directed, 'a')
print(dist)

{'a': 0, 'b': 1, 'c': 2, 'd': 2, 'e': 3, 'f': 3, 'g': 4}


## Floyd-Warshall algorithm for all-pairs shortest paths

Disktra's algorithm computes shortest paths from a single source to all other nodes. To compute shortest paths between all pairs of nodes we can use the algorithm proposed by Floyd and Warshall. Again, we provide an implementation (and brief explanation) only to refresh your memory from BSc courses. The Floyd-Warshall algorithm works as follows: It uses a distance matrix that stores the shortest path distance from each node to every other node. Initially, this matrix is filled with $\inf$, except for the diagonal, which contains zeros. Entries corresponding to pairs of nodes connected by directed or undirected links are set to one. 

The algorithm iterates through all nodes $v$. For each node $v$, it checks all source and destination nodes $s$ and $d$, and tests whether the currently known distance from $s$ to $d$ is larger than the sum of the distances from $s$ to $v$ and from $v$ to $d$. If this is the case, it updates the distance (in which case we have found a shorter path). Once the nested for loop has finished, all entries in the matrix correspond to the shortest path distances.

In [19]:
def floyd_warshall(network: pp.Graph):
    n = network.N

    m = network.mapping

    # Initialize distance matrix
    distance_matrix = np.zeros((n, n))
    distance_matrix.fill(np.inf)
    np.fill_diagonal(distance_matrix, 0)
    
    # set distances of pairs of nodes connected by (directed or undirected) edges
    for (v,w) in network.edges:
        distance_matrix[m.to_idx(v), m.to_idx(w)] = 1
        if not network.is_directed:
            distance_matrix[m.to_idx(w), m.to_idx(v)] = 1

    # update distances for all triples of nodes
    for v in network.nodes:
        for s in network.nodes:
            for d in network.nodes:
                if distance_matrix[m.to_idx(s), m.to_idx(d)] > distance_matrix[m.to_idx(s), m.to_idx(v)] + distance_matrix[m.to_idx(v), m.to_idx(d)]:
                    distance_matrix[m.to_idx(s), m.to_idx(d)] = distance_matrix[m.to_idx(s), m.to_idx(v)] + distance_matrix[m.to_idx(v), m.to_idx(d)]
    return distance_matrix

We apply the algorithm to the directed network from above. The vector in the first row corresponds to the distance vector that was returned by the Dijkstra algorithm for source node `a`. The fact that the distance matrix contains $\inf$ values shows that this directed network is not strongly connected, i.e. there are pairs of nodes for which no path exists. We will explore connected components in more detail in the next notebook.

In [20]:
m = floyd_warshall(n_directed)
print(m)

[[ 0.  1.  2.  2.  3.  3.  4.]
 [ 2.  0.  1.  1.  2.  2.  3.]
 [ 1.  2.  0.  3.  4.  4.  5.]
 [inf inf inf  0.  1.  1.  2.]
 [inf inf inf  3.  0.  1.  2.]
 [inf inf inf  2.  3.  0.  1.]
 [inf inf inf  1.  2.  2.  0.]]


## Floyd-Warshall algorithm with path reconstruction

The implementation of the Floyd-Warshall algorithm above calculates the distance matrix, but does not return actual shortest paths. We can add this feature by adding a matrix that stores a next pointer for each pair of source and target nodes (i.e. since we compute all shortest paths for all source nodes we have a next pointer matrix rather than a next pointer vector). This next pointer is updated whenever a new shortest path between a pair of nodes is found.

In [21]:
def floyd_warshall_paths(network: pp.Graph):
    n = network.N
    m = network.mapping

    # initialize distance matrix    
    distance_matrix = np.zeros((n, n))
    distance_matrix.fill(np.inf)
    np.fill_diagonal(distance_matrix, 0)

    # initialize next pointers to None
    next_pointer = defaultdict(lambda: dict())
    for s in network.nodes:
        for d in network.nodes:
            next_pointer[s][d] = None
    
    # set distances and next pointers for pairs of nodes connected by an edge
    for (v,w) in network.edges:
        distance_matrix[m.to_idx(v), m.to_idx(w)] = 1
        next_pointer[v][w] = w
        if not network.is_directed:
            distance_matrix[m.to_idx(w), m.to_idx(v)] = 1
            next_pointer[w][v] = v
    
    # update distances and next pointers
    for v in network.nodes:
        for s in network.nodes:
            for d in network.nodes:
                if distance_matrix[m.to_idx(s), m.to_idx(d)] > distance_matrix[m.to_idx(s), m.to_idx(v)] + distance_matrix[m.to_idx(v), m.to_idx(d)]:
                    distance_matrix[m.to_idx(s), m.to_idx(d)] = distance_matrix[m.to_idx(s), m.to_idx(v)] + distance_matrix[m.to_idx(v), m.to_idx(d)]
                    next_pointer[s][d] = next_pointer[s][v]
    return distance_matrix, next_pointer

We can now write a function that uses the next pointers to construct the shortest path from any source to any destination node.

In [22]:
def construct_path(next_pointer, s, d):
    # no path exists
    if next_pointer[s][d] == None:
        return tuple()
    # reconstruct path by traversing matrix of next pointers
    path = tuple(s)
    x = s
    while x != d:
        x = next_pointer[x][d]
        path += (x,)
    return path

Let us try this in the undirected network from above:

In [34]:
pp.plot(n_undirected, edge_color='white', node_label = [x for x in n_undirected.mapping.node_ids])

In [24]:
m, next_pointer = floyd_warshall_paths(n_undirected)
construct_path(next_pointer, 'g', 'e')

('g', 'd', 'e')

In [25]:
construct_path(next_pointer, 'a', 'e')

('a', 'b', 'd', 'e')

## Bellman-Ford algorithm for single-source shortest paths in weighted graphs

The algorithms introduced above can also be applied to weighted graphs. While Dijkstra's algorithm is limited to link with positive weights, the Floyd-Warshall algorithm can also be applied to networks with negative link weights (provided that those networks do not contain negative cycles). We now present an alternative single-source shortest path algorithm that can be applied to networks with positive and negative link weights. The Bellman-Ford algorithm is based on a similar idea like Dijkstra's algorithm. Starting from a state in which all distances are set to infinity, it repeatedly improves the current distance estimation based on the edge weights. Different from Dijkstra's algorithm, which uses a priority queue to select the next vertex to be processed, the update of distances is simply applied n-1 times for all edges. This makes the algorithm more versatile, i.e. it can be applied to networks with negative edge weights. However its worst-case computational complexity is in $\mathcal{O}(n\cdot m)$ rather than $\mathcal{O}(n+m\cdot \log n)$ for Dijkstra.

In [32]:
def BellmanFord(network: pp.Graph, source):
    distance = {}
    predecessor = {}

    # Step 1: initialize graph
    for v in network.nodes:
        # Initialize the distance to all vertices to infinity
        distance[v] = float("inf")
        # And having a null predecessor
        predecessor[v] = None         

    # The distance from the source to itself is, of course, zero
    distance[source] = 0              

    # Step 2: relax edges repeatedly    
    for i in range(0, network.N-1): 
        for (u,v) in network.edges:
            w = network['edge_weight', u, v].item()
            if distance[u] + w < distance[v]:
                distance[v] = distance[u] + w
                predecessor[v] = u

    # Step 3: check for negative-weight cycles
    for (u,v) in network.edges:
        w = network['edge_weight', u, v].item()
        if distance[u] + w < distance[v]:
            raise Exception("Graph contains a negative-weight cycle")

    return distance, predecessor

Let us try this in a small example network where all edge weights are one, except for one edge that has a negative edge weight:

In [35]:
import torch

n = pp.Graph.from_edge_list([('a','b'), ('b','c'), ('c','d'), ('c','e'), ('d','f')])
n.data.edge_weight = torch.tensor([1,1,-2,1,1])
pp.plot(n, edge_color='white', node_label = [x for x in n.mapping.node_ids])

In [33]:
distance, predecessors = BellmanFord(n, source='a')
print(distance)
print(predecessors['e'])

{'a': 0, 'b': 1, 'c': 2, 'd': 0, 'e': 3, 'f': 1}
c


Let us add an edge that forms a cycle of length two. The weight of this edge is three, such that a single traversal of this cycle has a total (positive) cost of one.

In [36]:
n = pp.Graph.from_edge_list([('a','b'), ('b','c'), ('c','d'), ('c','e'), ('d','f'), ('d', 'c')])
n.data.edge_weight = torch.tensor([1,1,-2,1,1,3])
pp.plot(n, edge_color='white', node_label = [x for x in n.mapping.node_ids])

This is an example for a network with negative link weight and a cycle. However, this is not a negative cycle because the total cost of traversing the cycle is positive. We can thus still apply the Bellman-Ford algorithm to find all shortest path from node `a` to all other nodes. We find that the cost of reaching node `e` is zero, but the cost of reaching node `f` is one.

In [37]:
distance, predecessors = BellmanFord(n, source='a')
print(distance)

{'a': 0, 'b': 1, 'c': 2, 'd': 0, 'e': 3, 'f': 1}


What happens if we reduce the weight of the link from `d` to `c`? Now traversing the cycle from `c` to `d` and back to `c` has a total cost of -1, i.e. we can infinitely reduce the cost by repeatedly traversing the cycle.

In [38]:
n = pp.Graph.from_edge_list([('a','b'), ('b','c'), ('c','d'), ('c','e'), ('d','f'), ('d', 'c')])
n.data.edge_weight = torch.tensor([1,1,-2,1,1,1])
pp.plot(n, edge_color='white', node_label = [x for x in n.mapping.node_ids])

The Bellman-Ford algorithm detects this negative cycle:

In [39]:
distance, predecessors = BellmanFord(n, source='a')
print(distance)
print(predecessors['e'])

Exception: Graph contains a negative-weight cycle

## Network diameter and average shortest path length

The calculation of shortest paths between all pairs of nodes is the basis to calculate diameter and average shortest path length of a network. We can simply compute the maximum and average of all entries in the distance matrix calculated by the Floyd-Warshall algorithm.

In [40]:
print('Diameter =', np.max(m))
print('Average shortest path length =', np.mean(m))

Diameter = 3.0
Average shortest path length = 1.5918367346938775


## Adjacency matrix powers, walks, and diameter

In the lecture, we have introduced the adjacency matrix, arguable the most fundamental matrix representation of a graph. In `pathpyG`, we can calculate the adjacency matrix of our undirected example graph as follows:

In [6]:
pp.plot(n_undirected, edge_color='white', node_label = [x for x in n_undirected.mapping.node_ids])

In [42]:
A = n_undirected.get_sparse_adj_matrix()
print(A)

  (0, 2)	1.0
  (0, 1)	1.0
  (1, 0)	1.0
  (1, 2)	1.0
  (1, 3)	1.0
  (2, 0)	1.0
  (2, 1)	1.0
  (3, 5)	1.0
  (3, 6)	1.0
  (3, 4)	1.0
  (3, 1)	1.0
  (4, 3)	1.0
  (4, 5)	1.0
  (5, 3)	1.0
  (5, 4)	1.0
  (5, 6)	1.0
  (6, 3)	1.0
  (6, 5)	1.0


To facilitate the handling of large sparse graphs with few edges, this function returns a sparse matrix representation that consists of the indices of non-zero entries in the binary matrix. To obtain a dense `numpy` matrix, we can write:

In [43]:
A.todense()

matrix([[0., 1., 1., 0., 0., 0., 0.],
        [1., 0., 1., 1., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 0.]], dtype=float32)

As explained in the lecture, the definition of matrix multiplication has the consequence that entries in the $k$-th power of the adjacency matrix correspond to the number of walks of length exactly $k$ between pairs of nodes. In our undirected toy example network, there are exactly two walks of length two from node `e` to node `g`, one passing via node `d` and one passing via node `f`. In this specific example, those two walks also correspond to the two shortest paths from `e` to `g`. Note however, that the entries of the adjacency matrix power do not only count (shortest) paths but all walks of a given length.

In [45]:
A_2 = (A**2).todense()
print(A_2)
print(A_2[n_undirected.mapping.to_idx('e'), n_undirected.mapping.to_idx('g')])

[[2. 1. 1. 1. 0. 0. 0.]
 [1. 3. 1. 0. 1. 1. 1.]
 [1. 1. 2. 1. 0. 0. 0.]
 [1. 0. 1. 4. 1. 2. 1.]
 [0. 1. 0. 1. 2. 1. 2.]
 [0. 1. 0. 2. 1. 3. 1.]
 [0. 1. 0. 1. 2. 1. 2.]]
2.0


We can actually use this property to calculate the diameter of a network based on the powers of the adjacency matrix. We sum $A^k$ for an increasing power $k$. The smallest $k$ at which the last zero entry in this sum disappears corresponds to the diameter of the network. This is because for this length $k$ we found at least one walk that connects the last pair of nodes. This (shortest) walk between the two nodes actually corresponds to the longest shortest path between any pair of nodes, which corresponds to the diameter of the network.

This algorithm can be implemented as follows:

In [46]:
def diameter(adj):
    """
        `adj' is adjacency matrix.

        Algorithm assumes that the network is strongly connected.

        Returns the diameter of the network.
    """
    # number of rows is the number of nodes
    n_nodes = adj.shape[0] 

    # every node has a walk to itself of length 0
    found_paths = np.identity(n_nodes) 

    # found_all is a boolean
    # True if we found walks between every pair of nodes 
    found_all = (found_paths>0).all() 

    # power
    k = 0 

    # adjacency matrix to the power k
    adj_k = np.identity(n_nodes) 

    while not found_all:
        #increase the power
        k += 1 

        #compute the power of the adjacency matrix
        adj_k = adj_k * adj 

        # add the walks of length k to the "found walks"
        found_paths = found_paths + adj_k 

        # check whether we found at least one
        # walk between each pair of nodes
        found_all = (found_paths>0).all() 

    return k

In [47]:
A = n_undirected.get_sparse_adj_matrix().todense()
diameter(A)

3

An interesting question that you can address in your self-studies is how the computational complexity of this matrix-based algorithm compares to that of the Floyd-Warshall algorithm, which we can use to calculate shortest paths between all pairs of nodes, and thus the diameter of the network. 